# Categorical Boost
#### cv score :  0.5639017013232513 (5 folds, LB 0.5609)
#### cv score :  0.5639017013232513 (5 folds, find rare, LB 0.5575)
#### cv score :  0.571281663516068 ... (LB 0.555) onehotなし、

In [1]:
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
import sklearn
from Functions import prepro
import warnings
warnings.filterwarnings('ignore')

import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
warnings.filterwarnings('ignore')
from catboost import Pool, CatBoostClassifier, CatBoost
import matplotlib.pyplot as plt
import seaborn as sns


FEATURE_SELECTION = False
tgt_encode = False

random.seed(0)

pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_rows", 1000)

%matplotlib inline

In [2]:
train_df = pd.read_csv("../../data/Processed/train2.csv")
test_df = pd.read_csv("../../data/Processed/test2.csv")
print(train_df.shape)
print(test_df.shape)
train_df.index = train_df.index -1
test_df.index = test_df.index -1

(66125, 304)
(28340, 303)


In [3]:
y = train_df["y"].values
train_df = train_df.drop("y", axis=1)

train_df["A1-level"] = train_df["A1-level"].astype(float)
test_df["A1-level"] = test_df["A1-level"].astype(float)
train_df["B1-level"] = train_df["B1-level"].astype(float)
test_df["B1-level"] = test_df["B1-level"].astype(float)

In [4]:
# add disconnection info

train_df = prepro.add_disconnection(train_df)
test_df = prepro.add_disconnection(test_df)

In [5]:
# add info about numeric column
# 武器データの中のnum_colsを持つ特徴について、チーム内の統計量を計算、levelは結構効いてる

# num_cols = ["level", "range", "rapid", "atack", "ink-sub", "fav-main", "good-special"]

num_cols = [
    "level", "range-main", "range-bullet-main", "distant-range_sub", 
    "rapid", "atack", "ink-sub", "fav-main", "good-special", "DPS", "kill_time_ika-main",
    "front_gap_human-main", "rensya_frame-main", "saidai_damege-main", "damage_min-sub", 
    "damage_max-sub", "install_num-sub", "good-sub", "damage_max-special", 
    "duration-special", "good-special", "direct_rad-special", "distant_rad-special"
]

cols = [col for col in train_df.columns if "A1" in col or "A2" in col or "A3" in col or "A4" in col or 
        "B1" in col or "B2" in col or "B3" in col or "B4" in col]
drop_cols = []
for col1 in cols:
    f = True
    for col2 in num_cols:
        if col2 in col1:
            f = False
    if f and train_df[col1].dtype in [int, float]:
        drop_cols.append(col1)
        
train_df = train_df.drop(columns=drop_cols)
test_df = test_df.drop(columns=drop_cols)
        
    

train_df = prepro.add_numeric_info(train_df, num_cols)
test_df = prepro.add_numeric_info(test_df, num_cols)



# 欠損値埋め先にやろう
欠損値がある列

rank, weapon, level ⇨　回線切れ or ナワバリ

weaponの欠損に並んでそれに関係ある列、A3, A4, B3, B4 ⇨ 回線切れ

level, weaponが消えていたら回線切れ

In [6]:
# rankの欠損値を埋める
train_df, test_df = prepro.fillna_rank(train_df, test_df)

#そのほかの欠損値を埋める
train_df, test_df = prepro.fillna(train_df, test_df)

In [7]:
print(train_df.isnull().sum().sum())
print(test_df.isnull().sum().sum())

0
0


# 特徴量エンジニアリング

In [8]:
# count reskin, by mode
print("reskin")
train_df, test_df = prepro.count_reskin(train_df, test_df)
#train_df, test_df = prepro.count_reskin_by_mode(train_df, test_df)

# count mainweapon, by mode
print("mainweapon")
train_df, test_df = prepro.count_mainweapon(train_df, test_df)
#train_df, test_df = prepro.count_mainweapon_by_mode(train_df, test_df)

# count subweapon, by mode
print("subweapon")
train_df, test_df = prepro.count_subweapon(train_df, test_df)
#train_df, test_df = prepro.count_subweapon_by_mode(train_df, test_df)

# count special, by mode
print("special")
train_df, test_df = prepro.count_special(train_df, test_df)
#train_df, test_df = prepro.count_special_by_mode(train_df, test_df)


#identify A1
train_df, test_df = prepro.identify_A1(train_df, test_df)


# 水増し, A1も統計量に含めた特徴を作る場合は水ましより先にやる
print("mizumashi")
train_df, y = prepro.mizumashi(train_df, y)

# is_nawabari
train_df, test_df = prepro.is_nawabari(train_df, test_df)

# match rank、単体で意味なし
train_df, test_df = prepro.match_rank(train_df, test_df)

# rankを二列に分ける
train_df, test_df = prepro.ranker(train_df, test_df)




reskin
mainweapon
subweapon
special
mizumashi


In [9]:
# add team info、メインはなくてもいい

train_df,  test_df = prepro.addTeamInfo(train_df, test_df, cols=["special", "subweapon", "category1", "category2", "mainweapon", "rank-mark"])
#train_df,  test_df = prepro.addTeamInfo(train_df, test_df, cols=["special", "subweapon", "category1", "category2"])

(160590, 548)
special-A
special-B
subweapon-A
subweapon-B
category1-A
category1-B
category2-A
category2-B
mainweapon-A
mainweapon-B
rank-mark-A
rank-mark-B
(160590, 752)
complete


In [10]:
# categorize team , 良い, メインはカテゴリ数が多すぎてやめた方がいい

categorize_col = ["category1", "category2", "subweapon", "special", "mainweapon"]
#categorize_col = ["category1", "category2", "subweapon", "special"]
for col in categorize_col:
    print(col)
    train_df, test_df = prepro.categorize_team(train_df, test_df, col)

category1
category2
subweapon
special
mainweapon


In [11]:
# レアなカテゴリをまとめる、しない方が上がった…

#train_df, test_df = prepro.find_rare(train_df, test_df, "team-category1")
#train_df, test_df = prepro.find_rare(train_df, test_df, "team-category2")
#train_df, test_df = prepro.find_rare(train_df, test_df, "team-subweapon")
#train_df, test_df = prepro.find_rare(train_df, test_df, "team-special")

In [12]:
# product categorical feature
"""train_df, test_df = prepro.prod(train_df, test_df, "mode", "stage")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-category1-A")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-category1-B")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-category2-A")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-category2-B")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-mainweapon-A")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-mainweapon-B")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-subweapon-A")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-subweapon-B")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-special-A")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-special-B")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "match_rank")"""

train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-category1-A")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-category1-B")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-category2-A")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-category2-B")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-mainweapon-A")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-mainweapon-B")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-subweapon-A")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-subweapon-B")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-special-A")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-special-B")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "match_rank")

"""
train_df, test_df = prepro.prod(train_df, test_df, "mode", "reskin-A1")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "reskin-A2")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "reskin-A3")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "reskin-A4")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "reskin-B1")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "reskin-B2")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "reskin-B3")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "reskin-B4")"""

'\ntrain_df, test_df = prepro.prod(train_df, test_df, "mode", "reskin-A1")\ntrain_df, test_df = prepro.prod(train_df, test_df, "mode", "reskin-A2")\ntrain_df, test_df = prepro.prod(train_df, test_df, "mode", "reskin-A3")\ntrain_df, test_df = prepro.prod(train_df, test_df, "mode", "reskin-A4")\ntrain_df, test_df = prepro.prod(train_df, test_df, "mode", "reskin-B1")\ntrain_df, test_df = prepro.prod(train_df, test_df, "mode", "reskin-B2")\ntrain_df, test_df = prepro.prod(train_df, test_df, "mode", "reskin-B3")\ntrain_df, test_df = prepro.prod(train_df, test_df, "mode", "reskin-B4")'

In [13]:
# make input

drop_cols = [
    "id", "lobby", "lobby-mode",  "period", "game-ver", "A1-weapon", "A2-weapon", "A3-weapon", "A4-weapon",
    "B1-weapon", "B2-weapon", "B3-weapon", "B4-weapon", "A-a-rank-mark-onehot", "A-o-rank-mark-onehot",
    "B-a-rank-mark-onehot", "B-o-rank-mark-onehot",
]

X, test_X = prepro.make_input(train_df, test_df, drop_cols, categorical_encode=False, scaler=False, verbose=False)

categorical_features_indices = np.where(X.dtypes == "object")[0]

complete


# 学習

In [ ]:
train_df["pred"] = 0
test_df["pred"] = 0
for mode in X["mode"].unique():
    print(mode)
    mode_index = X[X["mode"] == mode].index
    mode_X = X.iloc[mode_index].reset_index(drop=True)
    mode_y = y[mode_index]
    
    
    
    # 全データを5つに分割
    random.seed(random.randint(0, 10000))
    SIZE = X.shape[0]
    K = 5

    #folds = prepro.make_stratified_kfolds(X, y, K, shuffle=True)
    folds = prepro.make_stratified_kfolds(mode_X, mode_y, K, shuffle=True, random_state=random.randint(0, 10000))


    params = {
        "loss_function" : "Logloss",
        "eval_metric" : "Logloss",
        "iterations":2000,
        "learning_rate" : 0.05,
        "use_best_model": True,
        "random_seed":random.randint(0, 100000),
        'task_type': 'GPU',



    }

    THRESHOLD = 0.50
    models = []
    cv_scores = []
    temp = 0
    train_pred = []
    valid_ys = []

    all_indices = sum(folds, [])
    for i in range(K):
        print("======================== fold {} ========================".format(i+1))
        valid_indices = folds[i]
        train_indices = list(set(all_indices) - set(valid_indices))
        # print("train ", len(train_indices), " , valid ", len(valid_indices))


        train_X = mode_X.iloc[train_indices]
        train_y = mode_y[train_indices]
        valid_X = mode_X.iloc[valid_indices]
        valid_y = mode_y[valid_indices]




        train_data = Pool(train_X, train_y, cat_features=categorical_features_indices)
        valid_data = Pool(valid_X, valid_y, cat_features=categorical_features_indices)

        model = CatBoostClassifier(**params)


        model.fit(
            train_data,
            eval_set=valid_data,
            early_stopping_rounds=50,
            verbose=100,
            use_best_model=True,
        )

        pred = model.predict(valid_X, prediction_type='Probability')[:,1]
        train_pred.append(pred)
        pred = np.where(pred < THRESHOLD, 0, 1)

        temp += np.sum(pred) 

        score = accuracy_score(pred, valid_y)

        models.append(model)
        cv_scores.append(score)


    print("cv score : ", np.mean(cv_scores))    
    print("cv ratio : ", temp / SIZE)
    
    preds = []
    for i in range(K):
        model = models[i]
        pred = model.predict(test_X[test_X["mode"] == mode], prediction_type='Probability')[:,1]
        preds.append(pred)
        print(np.sum(pred) / pred.shape[0])

    preds = np.array(preds)
    preds = np.mean(preds, axis=0)
    print(np.sum(preds) / preds.shape[0])


    temp = pd.DataFrame({"pred":pred})

    #preds = np.where(preds < THRESHOLD, 0, 1)
    print(np.sum(preds) / preds.shape[0])


    mode_X["pred"] = 0
    for i in range(K):
        mode_X["pred"].iloc[folds[i]] = train_pred[i]
        
    train_df["pred"].iloc[mode_index] = mode_X["pred"].values
    test_df["pred"][test_X["mode"] == mode] = pred




======================== fold 1 ========================
0:	learn: 0.6929175	test: 0.6928460	best: 0.6928460 (0)	total: 238ms	remaining: 7m 55s
100:	learn: 0.6785735	test: 0.6832323	best: 0.6832323 (100)	total: 10.2s	remaining: 3m 12s
200:	learn: 0.6697885	test: 0.6799858	best: 0.6799858 (200)	total: 20.1s	remaining: 2m 59s
300:	learn: 0.6586291	test: 0.6786007	best: 0.6784483 (296)	total: 29.9s	remaining: 2m 48s
400:	learn: 0.6483435	test: 0.6774245	best: 0.6774120 (398)	total: 39.8s	remaining: 2m 38s
500:	learn: 0.6383979	test: 0.6760825	best: 0.6760189 (498)	total: 49.8s	remaining: 2m 28s
600:	learn: 0.6290004	test: 0.6752856	best: 0.6751006 (571)	total: 59.8s	remaining: 2m 19s
700:	learn: 0.6207870	test: 0.6747016	best: 0.6747016 (700)	total: 1m 9s	remaining: 2m 9s
bestTest = 0.674701589
bestIteration = 700
Shrink model to first 701 iterations.
======================== fold 2 ========================
0:	learn: 0.6928100	test: 0.6930204	best: 0.6930204 (0)	total: 148ms	remaining: 4m

300:	learn: 0.6557705	test: 0.6792211	best: 0.6791902 (298)	total: 29.9s	remaining: 2m 48s
400:	learn: 0.6444682	test: 0.6779666	best: 0.6778705 (390)	total: 39.8s	remaining: 2m 38s
500:	learn: 0.6340335	test: 0.6769609	best: 0.6767894 (491)	total: 49.8s	remaining: 2m 29s
bestTest = 0.6767894116
bestIteration = 491
Shrink model to first 492 iterations.
======================== fold 3 ========================
0:	learn: 0.6928190	test: 0.6928839	best: 0.6928839 (0)	total: 139ms	remaining: 4m 37s
100:	learn: 0.6790477	test: 0.6835027	best: 0.6835027 (100)	total: 10s	remaining: 3m 8s
200:	learn: 0.6707981	test: 0.6808710	best: 0.6807835 (187)	total: 19.9s	remaining: 2m 58s
300:	learn: 0.6587187	test: 0.6793454	best: 0.6790395 (271)	total: 29.9s	remaining: 2m 48s
400:	learn: 0.6465954	test: 0.6767896	best: 0.6767896 (400)	total: 39.9s	remaining: 2m 39s
500:	learn: 0.6371590	test: 0.6752160	best: 0.6751119 (484)	total: 50s	remaining: 2m 29s
600:	learn: 0.6277272	test: 0.6723168	best: 0.67229

In [ ]:
train_df["pred"] = 0
test_df["pred"] = 0
for mode in X["mode"].unique():
    mode_index = X[X["mode"] == mode].index
    mode_X = X.iloc[mode_index].reset_index(drop=True)
    mode_y = y[mode_index]
    
    mode_X["pred"] = 0
    for i in range(K):
        mode_X["pred"].iloc[folds[i]] = train_pred[i]
        
    train_df["pred"].iloc[mode_index] = mode_X["pred"].values
    test_df["pred"][test_X["mode"] == mode] = pred

In [ ]:
preds = np.where(preds < THRESHOLD, 0, 1)
print(np.sum(preds) / preds.shape[0])

submit_df = pd.DataFrame({'y': preds})
submit_df.index.name = 'id'
submit_df.to_csv('../Submissions/submission_cat_6_{}.csv'.format(K))

# モデル解釈

In [ ]:
importance = pd.DataFrame(
    (models[0].get_feature_importance() + 
    models[1].get_feature_importance() + 
    models[2].get_feature_importance() + 
    models[3].get_feature_importance() + 
    models[4].get_feature_importance())/5
    , index=X.columns, columns=['importance']).sort_values('importance', ascending=False)
display(importance[:120])

In [ ]:
indices = [index for index in importance.index if "team" in index]
importance.loc[indices]

In [ ]:
train_pred

In [ ]:
train_df[train_df["disconnection-A"] != 0][["disconnection-A", "disconnection-B", "y", "pred"]]

In [ ]:
train_df["pred"] = 0
train_df["y"] = y
for i in range(K):
    train_df["pred"].iloc[folds[i]] = np.where(train_pred[i] < THRESHOLD, 0, 1)
    #train_df["pred"].iloc[folds[i]] = train_pred[i]
for mode in train_df["mode"].unique():
    rate = train_df[train_df["mode"] == mode][train_df["pred"] == train_df["y"]].shape[0] / train_df[train_df["mode"] == mode].shape[0]
    print("{} : {}".format(mode, rate))
print(train_df[train_df["pred"] == train_df["y"]].shape[0] / train_df.shape[0])

In [ ]:
result_df = train_df[train_df["y"] != train_df["pred"]]

In [ ]:
print(result_df[result_df["mode"] == "nawabari"].shape[0]/result_df.shape[0])
print(result_df[result_df["mode"] == "hoko"].shape[0]/result_df.shape[0])
print(result_df[result_df["mode"] == "asari"].shape[0]/result_df.shape[0])
print(result_df[result_df["mode"] == "area"].shape[0]/result_df.shape[0])
print(result_df[result_df["mode"] == "yagura"].shape[0]/result_df.shape[0])

In [ ]:
result_df[result_df["mode"] == "nawabari"]

In [ ]:
preds

In [ ]:
def plot_feature_importance(importance,names,model_type, MAXSIZE=120):

    #Create arrays from feature importance and feature names
    feature_importance = np.array(importance)[:MAXSIZE]
    feature_names = np.array(names)[:MAXSIZE]

    #Create a DataFrame using a Dictionary
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)

    #Sort the DataFrame in order decreasing feature importance
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

    #Define size of bar plot
    plt.figure(figsize=(20,16))
    #Plot Searborn bar chart
    sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
    #Add chart labels
    plt.title(model_type + 'FEATURE IMPORTANCE')
    plt.xlabel('FEATURE IMPORTANCE')
    plt.ylabel('FEATURE NAMES')

In [ ]:
plot_feature_importance(importance.values.reshape(-1,), importance.index, "Cat Boost ", MAXSIZE=40)

In [ ]:
train_df.index = train_df.index -1

In [ ]:
X.iloc[X[X["mode"] == "nawabari"].index]

In [ ]:
train_df.reset_index